In [105]:
import pandas as pd
#https://www.data.gouv.fr/fr/datasets/open-medic-base-complete-sur-les-depenses-de-medicaments-interregimes/
df = pd.read_csv("./OPEN_MEDIC_2022.CSV",sep=";",encoding='latin-1')

In [106]:
atc_lib = pd.read_csv("./ATC5.csv",sep=";",encoding='latin-1')
region_lib = pd.read_csv("./region.csv",sep=";",encoding='latin-1')
sexe_lib = pd.read_csv("./sexe.csv",sep=";",encoding='latin-1')
age_lib = pd.read_csv("./age.csv",sep=";",encoding='latin-1')

In [107]:
boites_regions = df.loc[:,["ATC5","BEN_REG","BOITES","age","sexe"]]

boites_regions_lib = pd.merge(boites_regions,atc_lib,on='ATC5',how='inner')
boites_regions_lib = pd.merge(boites_regions_lib,region_lib,on='BEN_REG',how='inner')
boites_regions_lib = pd.merge(boites_regions_lib,age_lib,on='age',how='inner')
boites_regions_lib = pd.merge(boites_regions_lib,sexe_lib,on='sexe',how='inner')

boites_regions_lib.head()


,ATC5,BEN_REG,BOITES,age,sexe,lib_atc,lib_reg,Lib_age,lib_sexe
0,A01AA01,5,17,0,1,SODIUM FLUORURE,Régions et Départements d'outre-mer,0-19 ANS,MASCULIN
1,A01AA01,5,66,0,1,SODIUM FLUORURE,Régions et Départements d'outre-mer,0-19 ANS,MASCULIN
2,A01AA01,5,20,0,1,SODIUM FLUORURE,Régions et Départements d'outre-mer,0-19 ANS,MASCULIN
3,A01AA01,5,14,0,1,SODIUM FLUORURE,Régions et Départements d'outre-mer,0-19 ANS,MASCULIN
4,A01AA51,5,1114,0,1,SODIUM FLUORURE EN ASSOCIATION,Régions et Départements d'outre-mer,0-19 ANS,MASCULIN


In [108]:
boites_regions_lib_paracetamol = boites_regions_lib[boites_regions_lib['lib_atc'].str.contains('PARACETAMOL')] #que les prescriptions de paracetamol


In [109]:
map1 = boites_regions_lib_paracetamol.loc[:,['BEN_REG','BOITES','lib_reg']].groupby(by=['BEN_REG','lib_reg'],as_index=False).sum()
map1.head(20)

,BEN_REG,lib_reg,BOITES
0,5,Régions et Départements d'outre-mer,11650282
1,11,Ile-de-France,87451946
2,24,Centre-Val de Loire,20908811
3,27,Bourgogne-Franche-Comté,23447856
4,28,Normandie,30674032
5,32,Nord-Pas-de-Calais-Picardie,56131169
6,44,Alsace-Champagne-Ardenne-Lorraine,46997776
7,52,Pays de la Loire,28537635
8,53,Bretagne,25850538
9,75,Aquitaine-Limousin-Poitou-Charentes,51246229


In [120]:
pop_region = pd.read_csv("./pop_region.csv",sep=';')
pop_region.head(20)
map1_rapport = pd.merge(map1,pop_region,on="BEN_REG",how='inner')
map1_rapport["boites/habitant"] = map1_rapport["BOITES"]/map1_rapport["PTOT"]
map1_rapport.head(20)
map1_rapport.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEN_REG          12 non-null     int64  
 1   lib_reg          12 non-null     object 
 2   BOITES           12 non-null     int64  
 3   PTOT             12 non-null     int64  
 4   boites/habitant  12 non-null     float64
dtypes: float64(1), int64(3), object(1)
memory usage: 612.0+ bytes


In [132]:
import folium
geo_path = './regions.geojson'

# Créez une carte centrée autour de la France
m = folium.Map(location=[46.2276, 2.2137], zoom_start=6)

# Ajoutez les frontières des régions à la carte en utilisant le GeoJSON
folium.Choropleth(
    geo_data=geo_path,
    name='choropleth',
    data=map1_rapport,
    columns=['BEN_REG', 'boites/habitant'],
    key_on='feature.properties.code',  # Ici, 'nom' doit correspondre à l'attribut du GeoJSON qui correspond au nom de la région
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Nombre de boites de medicaments par region et par habitant'
).add_to(m)


# Convert the 'BEN_REG' column in 'map1_rapport' DataFrame to string to ensure matching types.
map1_rapport['BEN_REG'] = map1_rapport['BEN_REG'].astype(str)

# Create a dictionary to map 'BEN_REG' to 'boites/habitant'.
boites_habitant_dict = map1_rapport.set_index('BEN_REG')['boites/habitant'].to_dict()

# Update the GeoJSON features with the 'boites/habitant' data from 'map1_rapport'.
for feature in geojson_data['features']:
    code = feature['properties']['code']
    # Check if the code exists in the DataFrame and add the 'boites/habitant' value.
    if code in boites_habitant_dict:
        feature['properties']['boites/habitant'] = boites_habitant_dict[code]
    else:
        feature['properties']['boites/habitant'] = "Data not available"


# Add the GeoJSON layer with the updated features to the map.
folium.GeoJson(
    geojson_data,
    name='French Regions',
    style_function=lambda feature: {'color': 'blue', 'weight': 2},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['nom', 'boites/habitant'],
        aliases=['Region:', 'Boîtes par habitant:'],
        localize=True
    ),
    popup=folium.features.GeoJsonPopup(
        fields=['nom','boites/habitant'],
        aliases=['Region:', 'Boîtes par habitant:'],
        localize=True
    )
).add_to(m)

# Add layer control and display the map.
folium.LayerControl().add_to(m)


m.save('map.html')